# Delegation Unit Test
Performing verification of the delegation mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

loaded GraphQL_data/SIstakeDelegateds.csv.
loaded GraphQL_data/SIstakeLockeds.csv.
loaded GraphQL_data/SIstakeWithdrawns.csv.
loaded GraphQL_data/SIallocationCollecteds.csv.
loaded GraphQL_data/SIstakeDepositeds.csv.


Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
delegation_events_dict = delegation_events[0]
print("EXPECTED TRUTH FROM DELEGATION EVENTS:")
for timestep, delegation_event in delegation_events_dict.items():
    print(f"{timestep-1}, {delegation_event[0]['delegator']}, {delegation_event[0]['shares']}")
    


EXPECTED TRUTH FROM DELEGATION EVENTS:
7, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
11, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55222.500000000000000000
46, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 2895.065106115432857526
50, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 19300.434040769552383512
84, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 3753.469591583507658440


In [4]:
print("MODELED RESULTS")
for timestep, delegation_event in delegation_events_dict.items():
    print(f"{timestep-1}, {delegation_event[0]['delegator']}, {df.iloc[timestep-1].delegators[delegation_event[0]['delegator']].shares}")


MODELED RESULTS
7, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
11, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55222.50000000000000000000000
46, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 2895.065106115432869397008508
50, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 19272.51290285808967240708694
84, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 3742.700754592425693787934901


In [5]:
print("UNITTEST RESULTS")
for timestep, delegation_event in delegation_events_dict.items():
    print(f"Event: Timestep={timestep-1}, Delegator ID={delegation_event[0]['delegator']}, Shares Granted={delegation_event[0]['shares']}")
    print(f"Model: Timestep={timestep-1}, Delegator ID={delegation_event[0]['delegator']}, Shares Granted={df.iloc[timestep-1].delegators[delegation_event[0]['delegator']].shares}")
    print("Shares Equal?", delegation_event[0]['shares'] == df.iloc[timestep-1].delegators[delegation_event[0]['delegator']].shares)
    print()


UNITTEST RESULTS
Event: Timestep=7, Delegator ID=0x63f8168f665f79101a0e0da2313cd1391e1c5e27, Shares Granted=1094.500000000000000000
Model: Timestep=7, Delegator ID=0x63f8168f665f79101a0e0da2313cd1391e1c5e27, Shares Granted=1094.500000000000000000
Shares Equal? True

Event: Timestep=11, Delegator ID=0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, Shares Granted=55222.500000000000000000
Model: Timestep=11, Delegator ID=0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, Shares Granted=55222.50000000000000000000000
Shares Equal? True

Event: Timestep=46, Delegator ID=0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, Shares Granted=2895.065106115432857526
Model: Timestep=46, Delegator ID=0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, Shares Granted=2895.065106115432869397008508
Shares Equal? False

Event: Timestep=50, Delegator ID=0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, Shares Granted=19300.434040769552383512
Model: Timestep=50, Delegator ID=0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, Shares Granted=19272.

# Conclusion
The number of shares granted by delegation in the first two events match the output of the cadCAD model exactly and then the next 3 are extremely close, diverging gradually.